In [1]:
import pandas as pd

c:\Users\nitis\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\nitis\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(7)

,artist,song,link,text
57643,Ziggy Marley,G7,/z/ziggy+marley/g7_20531173.html,Seven richest countries in the world \r\nThem...
57644,Ziggy Marley,Generation,/z/ziggy+marley/generation_20531171.html,Many generation have passed away \r\nFighting...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Ray Boltz,Break The Yoke,"Standing up on the outside, \r\nFalling down ..."
1,Taylor Swift,Umbrella,"You had my heart, we'll never be worlds apart ..."
2,Young Jeezy,Nothing,[Chorus] \r\nMan them young niggas killing bo...
3,Yoko Ono,Angry Young Woman,"Angry young woman on sunrise strip, \r\nWalki..."
4,Deep Purple,Chasing Shadows,"Chasing shadows, over my walls \r\nwith mysel..."
5,Neil Diamond,Blue Destiny (Version Two),Once you were mine \r\nAnd you wanted the who...
6,Lloyd Cole,Perfect Skin,I choose my friends only far too well \r\nI'm...
7,Counting Crows,4 White Stallions,She had four white stallions coming up around ...
8,Cliff Richard,It's Alright Now,People came from miles around \r\nTo pay resp...
9,Gordon Lightfoot,Apology,I remember everything \r\nI've learned about ...


In [9]:
df['text'][0]

"Standing up on the outside,  \r\nFalling down within,  \r\nLooking righteous as a pharisee,  \r\nFilled with guilt and sin,  \r\nTrying to walk with Jesus  \r\nThe strait and narrow road,  \r\nHolding on to this world,  \r\nI just could not let go,  \r\nThat's such a heavy road.  \r\nChorus  \r\nCome and break the yoke Lord,  \r\nCome and set this child free,  \r\nCome tear down the walls of pride  \r\nAnd all I hide beneath,  \r\nFreedom always came to those  \r\nWho receive the words you spoke,  \r\nI receive your word, break the yoke,  \r\nI receive your word, break the yoke.  \r\nPraying with my brother  \r\nIs such a precious time,  \r\nWhen I confessed my weaknesses  \r\nHe joined his hands with mine,  \r\nI know something happened  \r\nWhen he prayed with me,  \r\nThe weight of sin fell off,  \r\nJesus set me free.  \r\nCome and break the yoke Lord,  \r\nCome and set your children free,  \r\nCome tear down the walls of pride  \r\nAnd false humility,  \r\nThe nail scars in your 

In [10]:
df.shape

(5000, 3)

Text Processing and Text Cleaning

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

C:\Users\nitis\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
text_data = df['text']

In [16]:
df['text'] = df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvector = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = tfidfvector.fit_transform(df['text'].astype(str))
similarity = cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.        , 0.01438715, 0.04066558, ..., 0.03941415, 0.01619914,
       0.02997359])

In [30]:
df[df['song'] == 'Apology']

,artist,song,text
9,Gordon Lightfoot,Apology,i rememb everyth i 've learn about surviv i 'v...


In [31]:
def recommendation(song_df, df, similarity):
    try:
        idx = df[df['song'] == song_df].index[0]
    except IndexError:
        print(f"Song '{song_df}' not found in the DataFrame.")
        return []

    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [32]:
recommended_songs = recommendation('Apology', df, similarity)
print(recommended_songs)

['Down To Earth', 'Fool To Think', 'Green River', 'Vicious', "Don't Walk Away", 'Gone Too Long', "Summer's Almost Gone", 'Words Of Wisdom', 'Love Is Gone', 'All For Love', "Don't Walk Away", "I've Done At Least One Thing", 'An Angel Came To Me', 'Holding Everything', 'I Was So Lucky', "It's Your Song", 'The Gift', 'God You Never Cried', 'Looking For Love', 'Ravine']
